# Efficiency of selected LDES vs Operation Cost Study

## 1. Import relevant modules and CEM.jl

In [ ]:
using CSV, DataFrames, JuMP, HiGHS
include("CEM.jl")

## 2. Load data

In [2]:
storageinfo = CSV.read(joinpath("data","storageinfo.csv"), DataFrame)
generatorinfo = CSV.read(joinpath("data","generatorinfo.csv"), DataFrame)
vreinfo = CSV.read(joinpath("data","vreinfo.csv"), DataFrame)
halfhourlydemand = CSV.read(joinpath("data","halfhourlydemand.csv"), DataFrame)
halfhourlyvrecf = CSV.read(joinpath("data","halfhourlyvrecf.csv"), DataFrame)
halfhourlydemand = halfhourlydemand[1:1488,:];

## 3. Get IDs of selected LDES (based on Case 2 - "all" scenario)

In [3]:
eff_results = Dict()

selected_ldes_ids = storageinfo[
    (storageinfo.name.==" Compressed_air_energy_storage") .| 
    (storageinfo.name.==" Aqueous_sulfur_flow_batteries") .| 
    (storageinfo.name.==" Vanadium_redox_flow_batteries") .|
    (storageinfo.name.==" Multijunction_photovoltaic_thermal_storage"), :id]

4-element Vector{Int64}:
  2
  9
 10
 11

## 4. Simulation

In [10]:
for s_id in selected_ldes_ids[1:4]
    eff_results[s_id] = []
    for eff in 50:5:95
        eff = eff / 100
        if storageinfo[s_id,:bestCaseEff] < eff
            storageinfo[s_id,:bestCaseEff] = eff
            sol = CEM(vreinfo, generatorinfo, storageinfo, halfhourlydemand, halfhourlyvrecf; 
                budget=30e9, voll=100e3, cd_rho=10, reserve="all", silent=true)
            push!(eff_results[s_id], (eff, sol.TOC))
        end
    end
    storageinfo = CSV.read(joinpath("cleaned","storageinfo.csv"), DataFrame)
end

In [ ]:
eff_results